In [ ]:
import numpy as np
import os, sys, re
from math import *
import importlib as imp
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from tekal import *
from uitintegratie import *
import pandas as pd
import dill

import numpy.ma as ma
import fit_beta as fb
import fit_weibull as fw
from gumplot import *
import scipy.special as scs
import bisect
from IPython.lib.deepreload import reload as dreload
import inspect
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import gumplot as gplt

In [ ]:
# Relevante herhaaltijden
Tretlist = [2,5,10,20,30,100,300,1250,3000,10000,30000,100000,400000]

<h3>Regressie HBV-to-SOBEK Rijn</h3>
Uit regressie is een relatie bepaald uit de HBV-(piek)afvoeren $Q_h$ en de bijbehorende Sobek (piek)afvoeren $Q_s$ als volgt (Henk vd Boogaard, <tt>regression_2018</tt>):
    $$Q_s\,=\,C\,+\,r_1Q_h\,+\,(r_2-r_1)\sigma_1\log\left(1+\exp\left(\frac{Q_h-\mu_1}{\sigma_1}\right)\right)$$
    $$\phantom{Q_s\,=\,C\,+\,r_1Q_h\,}+\,(r_3-r_2)\,\sigma_2\log\left(1+\exp\left(\frac{Q_h-\mu_2}{\sigma_2}\right)\right)$$    
    $$\phantom{Q_s\,=\,C\,+\,r_1Q_h\,}+\,(r_4-r_3)\,\sigma_3\log\left(1+\exp\left(\frac{Q_h-\mu_3}{\sigma_3}\right)\right)$$
    $$\phantom{Q_s\,=\,C\,+\,r_1Q_h\,}+\,(r_5-r_4)\,\sigma_4\log\left(1+\exp\left(\frac{Q_h-\mu_4}{\sigma_4}\right)\right)$$

In [ ]:
import math   # Crucial import !!!
def regression_2018(x, fit_regr=False):
    c      = 342.254390
    r1     = 0.91577902
    mu1    = 14442.2780 #ORIG
    sigma1 = 307.118620
    r2     = 0.32752076
    mu2    = 18573.2200 #ORIG
    sigma2 = 2.71804449E-14
    r3     = 0.53632634 #ORIG
    mu3    = 21904.1250 #ORIG
    sigma3 = 136.807970  
    r4     = 0.21296795
    mu4    = 26177.6500 #ORIG
    sigma4 = 79.5476640
    r5     = 0.00000000
    if fit_regr == True:
        mu1    = 14800.0000
        mu2    = 17750.0000
        r3     = 0.75000000
        mu3    = 20000.0000
    
    x_ = []
    for value in x:
        z1 = (value-mu1)/sigma1
        z2 = (value-mu2)/sigma2
        z3 = (value-mu3)/sigma3
        z4 = (value-mu4)/sigma4
        
        a1 = (r1*value)
        a2 = ((r2-r1)*sigma1*np.log(1+np.exp(z1)))
        try:
            a3 = ((r3-r2)*sigma2*np.log(1+math.exp(z2)))
        except:
            a3 = ((r3-r2)*sigma2*z2)
        a4 = ((r4-r3)*sigma3*np.log(1+np.exp(z3)))
        a5 = ((r5-r4)*sigma4*np.log(1+np.exp(z4)))
        
        y = c + a1 + a2 + a3 + a4 + a5
        x_.append(y)

    return x_

In [ ]:
# Tabel interpolatie
def interpolate(xx,yy,x,default=np.nan,fraction=-1):
    if x<xx[0] or x>xx[-1]:
        return default
    for i in range(len(xx)-1):
        if (x>=xx[i] and x<=xx[i+1]):
            if fraction<0.0:
                w=(x-xx[i])/(xx[i+1]-xx[i])   # use fraction=0.0 for yy[i] and fraction=1.0 for yy[i+1]
            else:
                w=fraction
            return (1.-w)*yy[i] + w*yy[i+1]

## Least-squares line fit
#def linfit(x,y):
#    nn = ma.count(x)
#    c = [(ma.sum(x*y)-ma.sum(x)*ma.sum(y)/nn)/(ma.sum(x*x)-ma.sum(x)*ma.sum(x)/nn)]
#    c.append((ma.sum(y)-c[0]*ma.sum(x))/nn)
#    return(c)

def linfit(xvec, yvec, wvec=None):
    if wvec is None:
        w = np.ones(len(xvec))
    else:
        w = wvec
    sw = ma.sum(w)
    sx = ma.sum(w*xvec)
    sy = ma.sum(w*yvec)
    sxx = ma.sum(w*xvec*xvec)
    sxy = ma.sum(w*xvec*yvec)   
    a = (sxy-sy*sx/sw)/(sxx-sx*sx/sw)
    b = (sy-a*sx)/sw
    return a,b

# lwrl based on a specified width
def lwrl(x,y,xtgt,width):
    pwr = 2.0
    xmax=np.max(x)
    xmin=np.min(x)
    ytgt = np.array([])
    for xx in xtgt:
        xxp = max(min(xx,xmax),xmin)
        mask = np.where(np.abs(x-xxp)<width)
        xsub = x[mask] - xxp
        ysub = y[mask]
        weights = (1.-(np.abs(xsub)/width)**pwr)**pwr
        a,b = linfit(xsub,ysub,weights)
        ytgt = np.append(ytgt,a*(xx-xxp) + b)
    return ytgt

def lower_exponential(x,a,b,lbnd,L):
    delta = (x-(L+lbnd-b)/a)
    if a*delta>0:
        y = a*x + b
    else:
        y = lbnd+L*np.exp(a/L*delta)
    return(y)

In [ ]:
# outintegration with linear extrapolation in terms of the reduced gumbel variate
def outintegrateExtrap(parms, levels, distfun, n_extpt, n_intpt):
    # extrapolates the discharge curve beyond the highest point with npt points
    # the additional points are equally spaced on the F-axis between the highest F and 1
    # the additional parametersets are obtained by loglinear extrapolation from the 
    # highest npoints points in the data on the F-axis
    nparam = len(parms['distpar'][0])

    # fit the linear coefficients for each of the parameters
    list_a = []
    list_b = []
    for ipar in range(nparam):
        y = np.array([partuple[ipar] for partuple in parms['distpar']][-n_intpt:])
        F = np.array(parms['F'][-n_intpt:])
#       logt = np.log(1./(1.-F))     # fit linearly to the logarithm of the return period
#       a,b = linfit(sgv,y)
        sgv = -np.log(-np.log(F))    # fit linearly to the standarised gumbel variate
        a,b = linfit(sgv,y)
        list_a.append(a)
        list_b.append(b)

    # add a tuple of parameters and the F-value for points equally spaced between F_last and 1.0
    fhighest = parms['F'][-2]
    for ipt in range(n_extpt):
        ff_ipt = fhighest + (1.-fhighest)/(n_extpt+1)*(ipt+1)
#       logt_ipt = np.log(1./(1.-ff_ipt))
#       parms['distpar'].append(tuple([list_a[ipar]*logt_ipt+list_b[ipar] for ipar in range(nparam)]))
        sgv_ipt = -np.log(-np.log(ff_ipt))
        parms['distpar'].append(tuple([list_a[ipar]*sgv_ipt+list_b[ipar] for ipar in range(nparam)]))
        parms['F'] = np.append(parms['F'],ff_ipt)
    cdf = outintegrate(parms, levels, distfun)
    return cdf[cdf<1], levels[cdf<1]


<H3> Referentie werklijn en normale onzekerheid voor de Rijn, HBV-resultaten voor een optimal slice</H3>

In [ ]:
# Lees de referentie werklijn voor de Rijn (2021)
with open('optimal_slice_mem6d_Fwet_500K_1951-2015_window61_part001-025_MAX.csv',"r") as fnin:
    qref = []
    minimum = 1000000.
    while True:
        line = fnin.readline()
        if not line:
            break
        syear, sqvalue = line.strip().split(',')
        try:
            year = int(syear)
            qvalue = float(sqvalue)
        except:
            continue
        bisect.insort(qref,qvalue)
nx = len(qref)
np_qref = np.array(qref)
np_sgv = -np.log(-np.log((np.array(list(range(nx)))+1.-0.3)/(nx+0.4)))

<H3> Onzekerheden (standaarddeviatie) behorende bij de werklijn: <br>
    Meteorologische (Jack-knife) en hydrologische (GLUE) onzekerheid gecombineerd</H3>

In [ ]:
# Lees TEKAL file met Meteo onzekerheden
listTEKALBlocks("Final_FreqCurve_Rhine_HBV.tek")
# Lees TEKAL file (Henk vd Boogaard, met HBV onzekerheden)
UncMeteo = readTEKAL('Final_FreqCurve_Rhine_HBV.tek','BL01',skip=99)
tbl_mu_Meteo = UncMeteo[3]
tbl_sigma_Meteo = UncMeteo[5]

# Voeg de geinterpoleerde sigma's toe als onzekerheid
sigma_Meteo = []
for i in tqdm(list(range(np_qref.size))):
    sigma_Meteo.append(interpolate(tbl_mu_Meteo,tbl_sigma_Meteo,np_qref[i],default=10))
np_sigma_Meteo = np.array(sigma_Meteo)

rijn_new={'sgv':np_sgv,'mu':np_qref,'sigma':np_sigma_Meteo}

<H3> Log-lineaire extrapolatie van de werklijn + onzekerheid van 1250 jaar naar langere herhalingstijden.</h3>

Ook een transformatie naar SOBEK afvoeren d.m.v Regression2018 is uitgevoerd, zie afbeelding. Het is belangrijk dat de extrapolatie van de werklijn en onzekerheid vóór deze transformatie plaatsvindt.
</H3>

In [ ]:
# Lineaire extrapolatie voor herhalingstijden boven de 1250 jaar
# van de 'HBV'- werklijn en de onzekerheid
Trmin = 1250 
Tr = 1./(1.-np.exp(-np.exp(-rijn_new['sgv'])))
mask = tuple([Tr>Trmin])
sgvmin = -np.log(-np.log(1.-1./Trmin))

a_mu, b_mu = linfit(rijn_new['sgv'][mask],rijn_new['mu'][mask])
a_sigma, b_sigma = linfit(rijn_new['sgv'][mask],rijn_new['sigma'][mask])

ymax = 20000 
yrange = 2000

sgvmax = 15
nsgv = 20
sgv_extrap = np.linspace(sgvmin,sgvmax,nsgv)
mu_extrap = sgv_extrap * a_mu + b_mu
sigma_extrap = sgv_extrap * a_sigma + b_sigma

mask = tuple([Tr<Trmin])
rijn_new_extrapolated = {}
rijn_new_extrapolated['sgv'] = np.concatenate((rijn_new['sgv'][mask], sgv_extrap))
rijn_new_extrapolated['mu'] = np.concatenate((rijn_new['mu'][mask], mu_extrap))
rijn_new_extrapolated['sigma'] = np.concatenate((rijn_new['sigma'][mask], sigma_extrap))

plt.plot(rijn_new_extrapolated['sgv'], rijn_new_extrapolated['mu'],'r-.',label='vóór Regression 2018')
plt.plot(rijn_new_extrapolated['sgv'], rijn_new_extrapolated['mu']+2*rijn_new_extrapolated['sigma'],'r+-', label='+/- 2xse')
plt.plot(rijn_new_extrapolated['sgv'], rijn_new_extrapolated['mu']-2*rijn_new_extrapolated['sigma'],'r+-')

plt.plot(rijn_new_extrapolated['sgv'], regression_2018(rijn_new_extrapolated['mu']),'b-.', label='ná Regression 2018')
plt.plot(rijn_new_extrapolated['sgv'], regression_2018(rijn_new_extrapolated['mu']+2*rijn_new_extrapolated['sigma']),'b+-', label='+/- 2xse')
plt.plot(rijn_new_extrapolated['sgv'], regression_2018(rijn_new_extrapolated['mu']-2*rijn_new_extrapolated['sigma']),'b+-')

ax=plt.gca()
fig=plt.gcf()
fig.set_size_inches(15,7)
ax.legend(loc="upper left")
gplt.add_gum_axis(Tretlist,ax,ymax,yrange)

<h3>Beschrijven van de hydraulische onzekerheid uit verstoorde SOBEK runs met de parameters van verdelingen</h3>

Voor elk van 50 events zijn 100 verschillende SOBEK simulaties gedraaid, elk met verstoorde parameters, 
om een maat te krijgen voor de onzekerheid in de hydraulische modellering (Latin-Hypercube). Voor elk van die 50 samples van elk 100 leden zijn hier de parameters geschat voor Beta, Normale en Uniforme verdeling. De Beta parameters zijn uiteindelijk niet gebruikt in deze berekening. Bij elke event hoort ook een referentie afvoer, dat is de SOBEK simulatie met onverstoorde parameters.

In [ ]:
# Load ensembles into dictionaries
# Read the file into a dictionary, labelled by event->timeseries
datafile = 'read_Qmax_sums_results.csv'
sep = ';'
with open(datafile,"r") as fnin:
    line = fnin.readline().strip()
    colnames = re.sub(r'[^\x00-\x7F]','', line).split(sep)
    ensembles = { colname:[] for colname in colnames}
    while line:
        line = fnin.readline()
        linesplit = line.strip().split(sep)
        for i in range(len(linesplit)):
            if linesplit[i]:
                try:
                    ensembles[colnames[i]].append(float(linesplit[i].replace(',','.')))
                except:
                    ensembles[colnames[i]].append(linesplit[i])

# Select realizations from ensembles:
first_realization = 0
last_realization = 100
refnr = 113

label2nr = {}
rownames = ensembles['Scenario']
for i in range(len(rownames)):
    label2nr[rownames[i]] = i
selected = ['Sum%d'%dd for dd in range (first_realization,last_realization+1)]    # selected labels for realizations
selnr = [label2nr[rowname] for rowname in selected]

# Determine min, max, alpha, beta for each ensemble
irow_ref = label2nr['Sum%d'%refnr]
a = np.array([])
b = np.array([])
alpha = np.array([])
beta = np.array([])
qref = np.array([])
mu = np.array([])
sigma = np.array([])
minvalue = np.array([])
maxvalue = np.array([])

a_2se = np.array([])
b_2se = np.array([])
alpha_2se = np.array([])
beta_2se = np.array([])
c0 = colnames.pop(0)
for icol in range(1,len(colnames)): 
    sample=np.array([ensembles[colnames[icol]][i] for i in selnr])
    est,var = fb.fitBetaMomentsJack(sample)
    
    alpha = np.append(alpha,est[0])
    beta = np.append(beta,est[1])
    a = np.append(a,est[2])
    b = np.append(b,est[3])
    alpha_2se = np.append(alpha,var[2])
    beta_2se = np.append(beta,var[3])
    a_2se = np.append(a,var[0])
    b_2se = np.append(b,var[1])
    qref = np.append(qref,ensembles[colnames[icol]][irow_ref])
    mu = np.append(mu,np.mean(sample))
    sigma = np.append(sigma,np.std(sample))
    minvalue = np.append(minvalue,np.min(sample))
    maxvalue = np.append(maxvalue,np.max(sample))
    
#sys.stdout.write("%11s    %8s %8s       %8s    %8s       %8s  %8s    %s\n"%('Ref','alpha','beta','A', 'B', 'mu', 'sigma', 'Label'))
#for i in range(len(qref)):
#   sys.stdout.write("%8.5f    %8.5f %8.5f    %8.5f %8.5f    %8.5f %8.5f    %s\n"%(qref[i],alpha[i],beta[i],a[i],b[i],mu[i],sigma[i],colnames[i]))    

sys.stdout.write("%11s    %8s %8s       %8s    %8s       %8s  %8s    %s\n"%('Ref','alpha','beta','A', 'B', 'min', 'max', 'Label'))
for i in range(len(qref)):
    sys.stdout.write("%8.5f    %8.5f %8.5f    %8.5f %8.5f    %8.5f %8.5f    %s\n"%(qref[i],alpha[i],beta[i],a[i],b[i],minvalue[i],maxvalue[i],colnames[i]))        

<h3>Beschrijven van de parameters als lineaire functies van de referentie afvoer</h3>

De boven afgeleide parameters van verdelingen voor elke event en bijbehorende referentie afvoer vormen een onregelmatig geheel. Om toch bruikbare parameters te krijgen voor een willekeurige referentie afvoer zijn lineaire verbanden gefit. 

In [ ]:
# linear fits on the location parameters of the Beta distribution A and B
c_a = linfit(qref,a)
c_b = linfit(qref,b)

# linear fits on the shape parameters alpha and beta
c_alpha = linfit(qref,alpha)
c_beta = linfit(qref,beta)

# linear fits on the normal parameters mu and sigma
c_mu = linfit(qref,mu)
c_sigma = linfit(qref,sigma)

# linear fits on the uniform parameters min and max
c_min = linfit(qref,minvalue)
c_max = linfit(qref,maxvalue)

In [ ]:
# gebaseerd op de uit LHS (SOBEK) exprimenten gevonden hydraulische onzekerheid
# geeft the verwachting en standaard deviatie van de SOBEK afvoer, als functie van de onverstoorde referentie afvoer
def hydraulic_uncertainty_uniform(qref):
    maxval_array = []
    minval_array = []
    for qr in qref:
        minval_array.append(-lower_exponential(qr,(c_mu[0]-c_min[0]), (c_mu[1]-c_min[1]),0,250)
                           + c_mu[0]*qr + c_mu[1])
        maxval_array.append(lower_exponential(qr,(c_max[0]-c_mu[0]), (c_max[1]-c_mu[1]),0,250)
                           + c_mu[0]*qr + c_mu[1])
    return (np.array(minval_array),np.array(maxval_array))
    

## gebaseerd op de uit LHS (SOBEK) exprimenten gevonden hydraulische onzekerheid
## geeft the verwachting en standaard deviatie van de SOBEK afvoer, als functie van de onverstoorde referentie afvoer
def hydraulic_uncertainty_normal(qref):
    mu_array = []
    sgm_array = []
    for qr in qref:
        sgm_array.append(lower_exponential(qr,c_sigma[0],c_sigma[1],0,250))
        mu_array.append(lower_exponential(qr,c_mu[0],c_mu[1],0,10))
    return (np.array(mu_array),np.array(sgm_array))
    
# Plot betrouwbaarheidsband voor de hydraulische onzekerheid symmetrisch (LHS experiment)
qq = np.linspace(100,18000,1000)
qq_mu, qq_sgm = hydraulic_uncertainty_normal(qq)
plt.plot (qq,qq_mu,'b-',label='gemiddelde')
plt.plot (qq,qq_mu+2*qq_sgm,'g--',label='2xse-interval')
plt.plot (qq,qq_mu-2*qq_sgm,'g--')

qq_minval, qq_maxval = hydraulic_uncertainty_uniform(qq)
plt.plot (qq,qq_minval,'r--',label='min, max')
plt.plot (qq,qq_maxval,'r--')

ax=plt.gca()
xlbl=ax.set_xlabel('Referentie afvoer m3/s')

fig=plt.gcf()
fig.set_size_inches(9,7)
lgnd=ax.legend(loc="upper left")
xg = ax.grid()

Hierboven een weergave van twee vormen van hydraulische onzekerheid: het minimum-maximum interval (uit grootste en kleinste waarde) <br>en het 2xse-interval. <br>
NB. <i>Alléén voor de normale verdeling is het 2xse-interval ongeveer gelijk aan het tweezijdig 95% interval, hier dus duidelijk niet. </i>

 <h3>Simulatie van hydrologische en hydraulische onzekerheid</h3>
 De combinatie van onzekerhed door simulatie (opeenvolgende trekkingen). <br>
 De gevolgde werkwijze voor elke standardised gumbel variate $y$, oftewel voor elke herhalingstijd in een set: 
 <ul>   
 <li>Interpoleer uit de HBV-gebaseerde extremenplot de bijbehorende waarde van de afvoer én de standaard deviatie
 <li>  Jackknife + hydrologsche model onzekerheid -> $\mu_{hbv}(y)$, $\sigma_{hbv}(y)$
 <li>  Trek met deze parameters een normaal sample
 <li>  Voor élke $q_{hbv}$ uit dat sample:
 <ul>   
     <li> Transformeer naar een 'SOBEK' waarde $q_{hbv} \rightarrow q_{sbk}$ met <tt>Regression2018</tt>
   <li> Als functie van deze $q_{sbk}$, haal de $\mu_{sbk}$ en $\sigma_{sbk}$ in het geval van een normale benadering, of de min en max in het geval van de uniforme benadering.<br> Hiervoor wordt de lineaire fit gebruikt, maar wel met restricties: $\sigma_{sbk}\ge0$ en ${\rm min}\le\mu_{sbk}\le{\rm max}$.
   <li>Trek met de parameters een nieuwe waarde $q_{sbk}'$<br>
     (deze waarde bevat het nu effect van de hydaulische onzekerheid)
 </ul> 
 <li> Uit het sample van $q_{sbk}'$ waarden, fit een 3-parameter weibull verdeling
 </ul>

In [ ]:
# Deze functie doet voor elke standardised gumbel variate y in de set stdgumvars het volgende: 
# * Interpoleert uit de HBV-gebaseerde extremenplot de bijbehorende waarde van de afvoer én de standaard deviatie
#   Jackknife + hydrologsche model onzekerheid -> mu_hbv, sigma_hbv
# * Trekt met de parameters mu_hbv en sigma_hbv een normaal sample
# * Voor élke q_hbv uit dat sample:
#   * Transformeer naar q_sobek
#   * als functie van deze q_sbk, haal de mu_sbk, sigma_sbk
#   * Trek met de parameters mu_sbk en sigma_sbk een nieuwe waarde q_sbk2
#     (deze waarde bevat het nu effect van de hydaulische onzekerheid)
# * Uit het sample van q_sbk2 waarden, fit een 3-parameter weibull verdeling

imp.reload(fw)

def sample_sbk_uniform(qsbk, sample_size):
    qq_minval, qq_maxval = hydraulic_uncertainty_uniform([qsbk])               # Bepaal normal parameters
    return np.random.uniform(low=qq_minval, high=qq_maxval, size=sample_size)   # trek hieruit een Normaal verdeelde sample

def sample_sbk_normal(qsbk, sample_size):
    qq_mu, qq_sgm = hydraulic_uncertainty_normal([qsbk])      # Bepaal normal parameters
    return np.random.normal(qq_mu, qq_sgm, sample_size)       # trek hieruit een Normaal verdeelde sample

def effective_sample_cascade(mu_hbv, sigma_hbv, sample_size, sample_func):
    # return the simulated sample of sobek discharges given the normal mu and sigma of the HBV-discharges only
    # this step takes into account the hydraulic uncertainty through the Beta distribution
    q_hbv = np.random.normal(mu_hbv,sigma_hbv,sample_size)# Trek een normaal sample met N(mu,sigma)
    q_sbk=regression_2018(q_hbv)                          # Transformeer dit sample met regression2018
    q_sbk0=regression_2018([mu_hbv])[0]                   # Transformeer gemiddelde met regression2018
    q_sbk2 = np.array([sample_func(qq,1)[0] for qq in q_sbk])
    return (q_sbk2)

def effective_weibull_random(stdgumvars,rijndata,sample_size,sample_func):
#   sample_size = 10000
    mu = rijndata['mu']
    sgv = rijndata['sgv']
    sgm = rijndata['sigma']
    qsamples = []
    mu_sbk = []
    for y in stdgumvars:
        qhbv_mu = interpolate(sgv,mu,y)               # verwachtingswaarde uit HBV voor de gegeven herhalingstijd
        qhbv_sigma = interpolate(sgv,sgm,y)           # stddev uit HBV voor de gegeven herhalingstijd
        qsample = effective_sample_cascade(qhbv_mu, qhbv_sigma, sample_size, sample_func)
        mu_sbk.append(regression_2018([qhbv_mu])[0])  # Transformeer de mu(hbv) om de werklijn te krijgen        
        qsamples.append(qsample)
       
    estimates = []
    for qs in qsamples:
        estimate = fw.fitWeibullLMoments(qs)
        # correct Location parameter C for the mean of the distribution    
        estimates.append(estimate)
    return (estimates, mu_sbk, qsamples)

Hier worden de simulaties aangeroepen voor zowel uiforme als normale hydraulische onzekerheid.<br>
Vervolgens is het 95%-betrouwbaarheids interval bepaald door de Weibull quantiel functie.

In [ ]:
sgvvals = np.concatenate((np.arange(-2,3.,step=0.2),np.arange(3.,15,step=0.5)))
size = 50000
estimates_uni, mu_sbk_uni, qsamples_uni = effective_weibull_random(sgvvals, rijn_new_extrapolated, size, sample_sbk_uniform)
q025_weibull_uniform = np.array([fw.qfunweibull(est[0],est[1],est[2],0.025) for est in estimates_uni])
q975_weibull_uniform = np.array([fw.qfunweibull(est[0],est[1],est[2],0.975) for est in estimates_uni])

estimates_norm, mu_sbk_norm, qsamples_norm = effective_weibull_random(sgvvals, rijn_new_extrapolated, size, sample_sbk_normal)
q025_weibull_normal = np.array([fw.qfunweibull(est[0],est[1],est[2],0.025) for est in estimates_norm])
q975_weibull_normal = np.array([fw.qfunweibull(est[0],est[1],est[2],0.975) for est in estimates_norm])

In [ ]:
ymax = 20000 
yrange = 2000

plt.plot(rijn_new_extrapolated['sgv'], rijn_new_extrapolated['mu'],'r-.',label='vóór Regression 2018')
plt.plot(rijn_new_extrapolated['sgv'], rijn_new_extrapolated['mu']+2*rijn_new_extrapolated['sigma'],'r+-', label='+/- 2xse')
plt.plot(rijn_new_extrapolated['sgv'], rijn_new_extrapolated['mu']-2*rijn_new_extrapolated['sigma'],'r+-')

plt.plot(rijn_new_extrapolated['sgv'], regression_2018(rijn_new_extrapolated['mu']),'b-.', label='ná Regression 2018')
plt.plot(rijn_new_extrapolated['sgv'], regression_2018(rijn_new_extrapolated['mu']+2*rijn_new_extrapolated['sigma']),'b+-', label='+/- 2xse')
plt.plot(rijn_new_extrapolated['sgv'], regression_2018(rijn_new_extrapolated['mu']-2*rijn_new_extrapolated['sigma']),'b+-')

plt.plot(sgvvals, q025_weibull_normal,'g--', label='mét hydraulische onzekerheid, normaal')
plt.plot(sgvvals, q975_weibull_normal,'g--')

plt.plot(sgvvals, q025_weibull_uniform,'m--', label='mét hydraulische onzekerheid, uniform')
plt.plot(sgvvals, q975_weibull_uniform,'m--')
plt.plot(sgvvals, mu_sbk_uni,'c--')

ax=plt.gca()
fig=plt.gcf()
fig.set_size_inches(15,7)
ax.legend(loc="upper left")
gplt.add_gum_axis(Tretlist,ax,ymax,yrange)

In [ ]:
# HydraRing parameters moeten gegeven zijn als functie van de referntie(=werklijn) afvoer, in dit geval het gemiddelde
# In de volgorde schaal, vorm, locatie voor de Weibull verdeling
# De referentie afvoer moet ervan zijn afgetrokken, zodat die door HydraRing kan worden bijgeteld

fpar = open('weibull_parameters_norm','w')
fpar.write("%10s, %10s, %10s, %10s\n"%('Qref', 'distpar1', 'distpar2', 'distpar3'))
sys.stdout.write("%10s, %10s, %10s, %10s\n"%('Qref', 'distpar1', 'distpar2', 'distpar3'))

nskip = 1
i = 0
for (mu_s,est) in zip(mu_sbk_norm,estimates_norm):
    i = i + 1
    if (i%(nskip+1)==0) or (i==np.size(mu_s)-1) or i==1:
        sys.stdout.write("%10.3f, %10.3f, %10.3f, %10.3f\n"%(mu_s,est[2],est[1],est[0]-mu_s))
        fpar.write("%10.3f, %10.3f, %10.3f, %10.3f\n"%(mu_s,est[2],est[1],est[0]-mu_s))
fpar.close()

sys.stdout.write('================================================================================'+"\n")
fpar = open('weibull_parameters_uniform','w')
fpar.write("%10s, %10s, %10s, %10s\n"%('Qref', 'distpar1', 'distpar2', 'distpar3'))
sys.stdout.write("%10s, %10s, %10s, %10s\n"%('Qref', 'distpar1', 'distpar2', 'distpar3'))

nskip = 1
i = 0
for (mu_s,est) in zip(mu_sbk_uni,estimates_uni):
    i = i + 1
    if (i%(nskip+1)==0) or (i==np.size(mu_s)-1) or i==1:
        sys.stdout.write("%10.3f, %10.3f, %10.3f, %10.3f\n"%(mu_s,est[2],est[1],est[0]-mu_s))
        fpar.write("%10.3f, %10.3f, %10.3f, %10.3f\n"%(mu_s,est[2],est[1],est[0]-mu_s))
fpar.close()

<h2>Scenarios</h2>

In [ ]:
infile =r"read_Qmax_sums_results.csv"

df = pd.read_csv(infile, sep=';', index_col=0)
df.dropna(inplace=True)

ref = df.loc[['Sum113']]

scenarios = ['Sum101','Sum102','Sum103',
             'Sum104','Sum105','Sum106',
             'Sum107','Sum108','Sum109',
             'Sum110','Sum111','Sum112',]

x_start = pd.Series(np.arange(250,12500,250))
y_start = pd.Series(np.arange(250,12500,250))

func = []
for scenario in scenarios:
    fig1, ax1 = plt.subplots(figsize=(12,6))
    scenA = df.loc[[scenario]]
    
    df2 = pd.concat([ref, scenA], axis=0).T
    x = df2['Sum113'].sort_values()
    y = df2[scenario].sort_values()
    
    x = pd.concat([x_start, x], axis=0)
    x.reindex()
    y = pd.concat([y_start, y], axis=0)
    
    z = np.polyfit(x, y, 3)
    f = np.poly1d(z)
    func.append(f)
    
    # calculate new x's and y's
    x_new = np.linspace(x.iloc[0], x.iloc[-1], 50)
    y_new = f(x_new)
    
    ax1.plot(x,y, 'ro', label=scenario)
    ax1.plot(x_new,y_new, 'r', label=scenario + "_fit")

    ax1.plot([0,20000],[0,20000], 'k--')  
    ax1.set_xlim(10000,20000)
    ax1.set_ylim(10000,)
    plt.legend()
    # plt.savefig(f"{scenario}_linfit.png", dpi=150, bbox_inches='tight')

In [ ]:
fig1, ax1 = plt.subplots(figsize=(12,6))

colors = ['blue','orange','red']
for scenario, color in zip(scenarios[0:3], colors):
    
    scenA = df.loc[[scenario]]
    
    df2 = pd.concat([ref, scenA], axis=0).T
    x = df2['Sum113'].sort_values()
    y = df2[scenario].sort_values()
    
    x = pd.concat([x_start, x], axis=0)
    x.reindex()
    y = pd.concat([y_start, y], axis=0)
    
    z = np.polyfit(x, y, 5)
    f = np.poly1d(z)
    func.append(f)
    
    # calculate new x's and y's
    x_new = np.linspace(x.iloc[0], x.iloc[-1], 50)
    y_new = f(x_new)
    
    ax1.scatter(x,y, marker = 'o', color=color, label=scenario)
    ax1.plot(x_new,y_new, color=color, label=scenario + "_fit")

ax1.plot([0,20000],[0,20000], 'k--')  
ax1.set_xlim(10000,20000)
# ax1.set_ylim(13500,20000)
# ax1.set_xlim(0,20000)
ax1.set_ylim(10000,)
plt.legend()
#plt.savefig(f"{scenario}_allinone.png", dpi=150, bbox_inches='tight')

In [ ]:
#plot oud versus observations
fig, ax = plt.subplots(1,1)
ax.set_xlim(0.325,12)
ymax = 20000 
yrange = 500
ax.set_ylim(0,ymax)

ax.set_xlim(8,12)
ax.set_ylim(14000,17000)

scenarios = ['Sum101','Sum102','Sum103']

for scenario in scenarios:
#scenario = 'Sum102'

    scenA = df.loc[[scenario]]

    df2['Sum113']

    df2 = pd.concat([ref, scenA], axis=0).T
    x = df2['Sum113'].sort_values()
    y = df2[scenario].sort_values()

    x = pd.concat([x_start, x], axis=0)
    x.reindex()
    y = pd.concat([y_start, y], axis=0)

    z = np.polyfit(x, y, 5)
    f = np.poly1d(z)
    func.append(f)

    np_mean_new = f(np_mean)

    ax.plot(np_yy[np_mean_new>13000],np_mean_new[np_mean_new>13000],linestyle='--',label="GRADE'21 - %s" %(scenario))

ax.plot(np_yy,np_mean,'b-',label="GRADE'21")
ax.plot(rijn_obs['sgv'],rijn_obs['mu'],'g*',label="Observations")

ax=plt.gca()
fig=plt.gcf()
fig.set_size_inches(15,7)
ax.legend(loc="upper left")
add_gum_axis(Tretlist,ax,ymax,yrange)
plt.savefig('plots//plot_10_all_zoom.png', dpi=150, bbox_inches='tight')


<hr>
<hr>